# PART 1 -Decision Tree classification

In [15]:
import pandas as pd
import numpy as np
import time

In [16]:
import numpy as np

class Node:
    def __init__(self, data, target):
        self.data = data
        self.target = target
        self.left = None
        self.right = None
        self.feature_index = None
        self.threshold = None
        self.value = None

def entropy(y):
    classes, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    return -np.sum(probabilities * np.log2(probabilities + 1e-10))  # Adding a small epsilon to avoid log(0)

def information_gain(y, y_left, y_right):
    H_parent = entropy(y)
    H_left = entropy(y_left)
    H_right = entropy(y_right)

    p_left = len(y_left) / len(y)
    p_right = len(y_right) / len(y)

    IG = H_parent - (p_left * H_left + p_right * H_right)
    return IG

def split_dataset(X, y, feature_index, threshold):
    left_mask = X[:, feature_index] <= threshold
    right_mask = ~left_mask
    return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

def find_best_split(X, y):
    m, n = X.shape
    if m <= 1:
        return None, None, None, None

    num_classes = len(set(y))
    best_ig = 0
#     print(best_ig)
    best_feature_index = None
    best_threshold = None

    for feature_index in range(n):
        
        thresholds = sorted(set(X[:, feature_index]))
        for i in range(1, len(thresholds)):
            threshold = (thresholds[i - 1] + thresholds[i]) / 2
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature_index, threshold)

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            ig = information_gain(y, y_left, y_right)
            
            if ig > best_ig:
#                 print(ig)
                best_ig = ig
                best_feature_index = feature_index
                best_threshold = threshold
#     print(best_feature_index,best_threshold)
    return best_feature_index, best_threshold

def build_tree(X, y, depth=0, max_depth=3):
    if depth == max_depth or len(set(y)) == 1:
        leaf = Node(data=None, target=y)
        leaf.value = max(set(y), key=list(y).count)
        return leaf

    feature_index, threshold = find_best_split(X, y)
    if feature_index is None:
        leaf = Node(data=None, target=y)
        leaf.value = max(set(y), key=list(y).count)
        return leaf
    X_left, y_left, X_right, y_right = split_dataset(X, y, feature_index, threshold)
    node = Node(data=(feature_index, threshold), target=None)
    node.left = build_tree(X_left, y_left, depth + 1, max_depth)
    node.right = build_tree(X_right, y_right, depth + 1, max_depth)

    return node


In [17]:
def predict(tree, x):
    if tree.value is not None:
        # If the current node is a leaf node, return its predicted value
        return tree.value
    else:
        # If the current node is not a leaf node, traverse to the appropriate child
        feature_index, threshold = tree.data
        if x[feature_index] <= threshold:
            return predict(tree.left, x)
        else:
            return predict(tree.right, x)


In [18]:
def smallest_prime_divisor(threshold, number):
        if threshold >= number:
            return 
        current_number = max(threshold + 1, 2)  # Ensure we start from at least 2
        while current_number < number:
            if  number % current_number == 0:
                return current_number
            current_number += 1
        return

In [19]:
def cross_validation(X,y):
        
        indices = np.arange(len(y))

        np.random.shuffle(indices)

        #compute the fold size
        fold_size =  int(len(y)/smallest_prime_divisor(10,len(y)))

        #determime folds
        folds = [indices[i:i+smallest_prime_divisor(10,len(y))] for i in range(0, len(y), smallest_prime_divisor(10,len(y)))]
        return folds,fold_size

In [20]:
def setconfusion_matrix(predicted_label,true_label):
    
        confusion_matrix = np.zeros((2, 2))
        for i in range(len(predicted_label)):
            if predicted_label[i] == 1 and true_label[i] == 1:
                confusion_matrix[1, 1] += 1  # True Positive
            elif predicted_label[i] == 1 and true_label[i] == 0:
                confusion_matrix[1, 0] += 1  # False Positive
            elif predicted_label[i] == 0 and true_label[i] == 1:
                confusion_matrix[0, 1] += 1  # False Negative
            elif predicted_label[i] == 0 and true_label[i] == 0:
                confusion_matrix[0, 0] += 1  # True Negative
#         print(confusion_matrix)
        return confusion_matrix
def compute_matrix(confusion_matrix):
    accuracy = (confusion_matrix[0, 0] + confusion_matrix[1, 1]) / np.sum(confusion_matrix)
    precision = confusion_matrix[1, 1] / (confusion_matrix[1, 0] + confusion_matrix[1, 1])
    recall = confusion_matrix[1, 1] / (confusion_matrix[0, 1] + confusion_matrix[1, 1])
    return accuracy,precision,recall
def calculate_average_metrics(accuracy_scores,precision_scores,recall_scores):
    # Calculate the average metrics
    average_accuracy = np.mean(accuracy_scores)
    average_precision = np.mean(precision_scores)
    average_recall = np.mean(recall_scores)
    
    return average_accuracy,average_precision,average_recall

In [21]:
dataset = pd.read_csv("trial.csv")
# dataset_ = pd.read_csv("hour.csv")
df_numeric = dataset.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with 0 
dataset = df_numeric.fillna(0)
# Extract features (X) by selecting all columns except the last one (labels)
X = dataset.iloc[:, :-1].values

# Extract labels (y) by selecting the last column
y = dataset.iloc[:, -1].values

In [22]:
folds,fold_size=cross_validation(X,y)

In [23]:
recall_scores=[]

precision_scores=[]

accuracy_scores=[]
start_time = time.time()
for i in range(fold_size):
    
    test_data = folds[i]

    train_data = np.concatenate([f for j, f in enumerate(folds) if j != i])
    
    X_train,y_train=X[train_data],y[train_data]
    
    dt_model=build_tree(X_train,y_train,0,3)
    
    X_test,y_test=X[test_data],y[test_data]
    
    predicitons=[predict(dt_model, sample) for sample in X_test]
    
    predictions=np.array(predicitons)

    confusion_matrix=setconfusion_matrix(predictions,y_test)
    print(f"Confusion matrix of the folds {i+1}")
    print(confusion_matrix)
    accuracy,precision,recall=compute_matrix(confusion_matrix)
    print(f"\naccuracy={accuracy},precision={precision},recall={recall} of the folds {i+1}\n")
    accuracy_scores.append(accuracy)
    
    precision_scores.append(precision)
    
    recall_scores.append(recall)

average_accuracy,average_precision,average_recall=calculate_average_metrics(accuracy_scores,precision_scores,recall_scores)
end_time = time.time()

# Calculate the elapsed time
totTime = end_time - start_time   
print(f"\naverage of the accuracy={average_accuracy},precision={average_precision},recall={average_recall} \n")
print(f"Total run time is {totTime}")

Confusion matrix of the folds 1
[[38.  0.]
 [ 0. 59.]]

accuracy=1.0,precision=1.0,recall=1.0 of the folds 1

Confusion matrix of the folds 2
[[41.  0.]
 [ 0. 56.]]

accuracy=1.0,precision=1.0,recall=1.0 of the folds 2

Confusion matrix of the folds 3
[[40.  0.]
 [ 0. 57.]]

accuracy=1.0,precision=1.0,recall=1.0 of the folds 3

Confusion matrix of the folds 4
[[33.  0.]
 [ 0. 64.]]

accuracy=1.0,precision=1.0,recall=1.0 of the folds 4

Confusion matrix of the folds 5
[[32.  0.]
 [ 0. 65.]]

accuracy=1.0,precision=1.0,recall=1.0 of the folds 5

Confusion matrix of the folds 6
[[37.  0.]
 [ 0. 60.]]

accuracy=1.0,precision=1.0,recall=1.0 of the folds 6

Confusion matrix of the folds 7
[[32.  0.]
 [ 0. 65.]]

accuracy=1.0,precision=1.0,recall=1.0 of the folds 7

Confusion matrix of the folds 8
[[37.  0.]
 [ 0. 60.]]

accuracy=1.0,precision=1.0,recall=1.0 of the folds 8


average of the accuracy=1.0,precision=1.0,recall=1.0 

Total run time is 1.427931785583496


# RESULT OF THE DECISION TREE BASE ON THE MODEL CLASSIFICATION

--> I have implemented decision tree for the classification. Result of the this model is very well.Therefore I couldnt trust result
. Then I have check attribute and threshold which are vest information gain. next I convert some data point to TP and FN. Model could find them and result have been change with these change. 

# PART 2 -Decision Tree REGRESSION

In [28]:
def sum_squared_residuals(y):
    if len(y) == 0:
        return 0
    mean = np.mean(y)
    return np.sum((y - mean) ** 2)

def total_squared_residuals(y_left, y_right):
    return sum_squared_residuals(y_left) + sum_squared_residuals(y_right)
def find_best_split_reg(X, y):
    m, n = X.shape
    if m <= 1:
        return None, None, None, None

    best_mse = float('inf')
    best_feature_index = None
    best_threshold = None

    for feature_index in range(n):
        thresholds = sorted(set(X[:, feature_index]))
        for i in range(1, len(thresholds)):
            threshold = (thresholds[i - 1] + thresholds[i]) / 2
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature_index, threshold)

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            mse = total_squared_residuals(y_left, y_right)

            if mse < best_mse:
                best_mse = mse
                best_feature_index = feature_index
                best_threshold = threshold

    return best_feature_index, best_threshold

def build_tree_reg(X, y, depth=0, max_depth=None):
    if depth == max_depth or len(set(y)) == 1:
        leaf = Node(data=None, target=y)
        leaf.value = np.mean(y)
        return leaf

    feature_index, threshold = find_best_split_reg(X, y)

    if feature_index is None:
        leaf = Node(data=None, target=y)
        leaf.value = np.mean(y)
        return leaf

    X_left, y_left, X_right, y_right = split_dataset(X, y, feature_index, threshold)

    node = Node(data=(feature_index, threshold), target=None)
    node.left = build_tree(X_left, y_left, depth + 1, max_depth)
    node.right = build_tree(X_right, y_right, depth + 1, max_depth)

    return node

In [29]:
dataset_ = pd.read_csv("hour.csv")
df_numeric_ = dataset_.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with 0 
dataset_ = df_numeric_.fillna(0)
# Extract features (X) by selecting all columns except the last one (labels)
X_ = dataset_.iloc[:, :-1].values

# Extract labels (y) by selecting the last column
y_ = dataset_.iloc[:, -1].values


In [30]:
folds_,fold_size_=cross_validation(X_,y_)

In [31]:
start_time_ = time.time()
mse_scores=[]
for i in range(fold_size_):
    start_time_1 = time.time()
    test_data = folds_[i]

    train_data = np.concatenate([f for j, f in enumerate(folds_) if j != i])
    
    X_train,y_train=X_[train_data],y_[train_data]
    
    dt_model=build_tree_reg(X_train,y_train,0,2)
    
    X_test,y_test=X_[test_data],y_[test_data]

    predicitons=[predict(dt_model, sample) for sample in X_test]  
    predictions=np.array(predicitons)
    mse = np.mean((predictions - y_test) ** 2)
    print(f"TEST FOLD {i+1}")
    print(f"MSE(Mean Squared Error) = {mse}")
    mse_scores.append(mse)
    end_time_1 = time.time()
    totTime_1 = end_time_1 - start_time_1   
    print(f"Run time of Fold {i+1}")
    print(totTime_1)
    
    
end_time_ = time.time()

# Calculate the elapsed time
totTime_ = end_time - start_time   
average_mse = np.mean(mse_scores)
print(f"Average MSE of DECISION TREE = {average_mse}")
print(f"Total run time is {totTime}")

TEST FOLD 1
MSE(Mean Squared Error) = 10094.382185396167
Run time of Fold 1
37.54422330856323
TEST FOLD 2
MSE(Mean Squared Error) = 9470.734334541688
Run time of Fold 2
37.43557286262512
TEST FOLD 3
MSE(Mean Squared Error) = 9230.970999482133
Run time of Fold 3
39.77035474777222
TEST FOLD 4
MSE(Mean Squared Error) = 8970.467115484205
Run time of Fold 4
39.1108820438385
TEST FOLD 5
MSE(Mean Squared Error) = 8747.392024857587
Run time of Fold 5
39.10930776596069
TEST FOLD 6
MSE(Mean Squared Error) = 9286.211807353702
Run time of Fold 6
40.92438578605652
TEST FOLD 7
MSE(Mean Squared Error) = 9717.697566027964
Run time of Fold 7
41.14089107513428
TEST FOLD 8
MSE(Mean Squared Error) = 7199.672190574832
Run time of Fold 8
38.20404577255249
TEST FOLD 9
MSE(Mean Squared Error) = 7946.398757120663
Run time of Fold 9
37.387815713882446
Average MSE of DECISION TREE = 8962.65855342655
Total run time is 1.427931785583496


# RESULT OF THE DECISION TREE BASE ON THE MODEL REGRESSION

In this model I Have implemented decision tree for the regression problem.I aim to find to best split and because of that I have used SSR(Sum Squared Residuals). I have used TSE(Total Squared Error) but its results are worst than SSR. Therefore I have chosen SSR in my model.My dataset it so big for classification. So That my run time is too bad.

# Implementation Random Forest